In [1]:
import numpy as np
from pyspark.sql import SparkSession

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1558400839934_0001,pyspark,idle,Link,Link,✔


SparkSession available as 'spark'.


In [2]:
spark = SparkSession \
    .builder \
    .appName("assignment2 stage 1 and stage 2") \
    .getOrCreate()

VBox()

In [3]:
rev_data = "s3://amazon-reviews-pds/tsv/amazon_reviews_us_Music_v1_00.tsv.gz"
revs = spark.read.csv(rev_data,header=True,sep='\t')

VBox()

In [13]:
#delete all null or None reviews
revs_remove_null = revs.where(revs['review_body'].isNotNull())

VBox()

# Stage 1 Overall Satistics

In [22]:
#the total number of reviews
total_number_revs = revs_remove_null.count()
print('the total number of reviews is: ', total_number_revs)
#the number of unique users
unique_user_revs = revs_remove_null.select('customer_id').distinct().count()
print('the number of unique users is: ', unique_user_revs)
#the number of unique products
unique_product_revs = revs_remove_null.select('product_id').distinct().count()
print('the number of unique products is: ', unique_product_revs)

VBox()

the total number of reviews is:  4751191
the number of unique users is:  1940631
the number of unique products is:  782281

In [23]:
#user-review distribution
group_user = revs_remove_null.groupBy('customer_id').count()
#largest number of reviews
largest_rev_user = group_user.describe('count').filter("summary = 'max'").select("count").collect()[0].asDict()['count']
print('the largest number of reviews for a user: ',largest_rev_user)
#top 10 users ranked by number of reviews they published
print('top 10 users ranked by reviews they published')
group_user.sort('count', ascending=False).show(10)
#median number of review a user has
median_user = group_user.approxQuantile('count', [0.5], 0)
print('the median number of reviews published by a user is ',median_user)

VBox()

the largest number of reviews for a user:  7168
top 10 users ranked by reviews they published
+-----------+-----+
|customer_id|count|
+-----------+-----+
|   50736950| 7168|
|   38214553| 5411|
|   51184997| 5369|
|   18116317| 4221|
|   23267387| 4023|
|   50345651| 3792|
|   14539589| 2896|
|   15725862| 2842|
|   19380211| 2592|
|   20018062| 2568|
+-----------+-----+
only showing top 10 rows

the median number of reviews published by a user is  [1.0]

In [24]:
#product-review distribution
group_product = revs_remove_null.groupBy('product_id').count()
#largest number of reviews
largest_rev_product = group_product.describe('count').filter("summary = 'max'").select("count").collect()[0].asDict()['count']
print('the largest number of reviews for single product: '+largest_rev_product)
#top 10 products ranked by number of review they have
print('top 10 products ranked by the number of reviews they have')
group_product.sort('count', ascending=False).show(10)
#median number of review a product has
median_product = group_product.approxQuantile('count', [0.5], 0)
print('the median number of reviews a product has ',median_product)

VBox()

the largest number of reviews for single product: 3936
top 10 products ranked by the number of reviews they have
+----------+-----+
|product_id|count|
+----------+-----+
|B00008OWZG| 3936|
|B0000AGWEC| 3326|
|B00MIA0KGY| 2699|
|B00NEJ7MMI| 2419|
|B000089RVX| 2376|
|B004EBT5CU| 2106|
|B0026P3G12| 2080|
|B00009PRZF| 2026|
|B00004XONN| 1901|
|B00006J6VG| 1793|
+----------+-----+
only showing top 10 rows

the median number of reviews a product has  [2.0]

# Stage 2 Filtering unwanted data

In [25]:
revs_require_column = revs_remove_null.select('customer_id','review_id','product_id','review_body')

VBox()

In [27]:
import re
from pyspark.sql.functions import udf
from pyspark.sql.functions import lit
from pyspark.sql.types import *
from pyspark.sql import functions as F

pattern = re.compile(r'[\.\?\!]+ ')
def countSentence(x):
    try:
        if x==None:
            num = 0
        else:
            num = len(re.split(pattern ,x)) # num is the number of sentence
        return num
    except Exception:
        return None
    
countSentences = udf(lambda x: countSentence(x), IntegerType())

#calculate the number of sentences a review body has
calculate_sentence = revs_require_column.withColumn("sNum", lit(countSentences(revs_require_column['review_body'])))

VBox()

In [41]:
from nltk.tokenize import sent_tokenize
def countSentence_nlp(x):
    if x==None:
        num = 0
    else:
        num = len(sent_tokenize(x))
    return num
    
countSentences_nlp = udf(lambda x: countSentence(x), IntegerType())

#calculate the number of sentences a review body has
calculate_sentence = revs_require_column.withColumn("sNum", lit(countSentences_nlp(revs_require_column['review_body'])))

VBox()

### Filter method 1
First use the origin dataframe to filter all reviews with two or more sentences and get the result A. </br>
Second use the result A to filter all reviews published by users with more than median number of reviews and get the result B</br>
Third use the result B to filter all reviews from product with more then median number of reviews and get the final result

In [42]:
rev_sen_filter = calculate_sentence.filter("sNum > 1").cache()

VBox()

In [52]:
median_user = 1.0
group_user = rev_sen_filter.groupBy('customer_id').count().withColumnRenamed("customer_id", "customer_id_group").withColumnRenamed("count", "customern")
revs_user_join1 = rev_sen_filter.join(group_user, rev_sen_filter.customer_id == group_user.customer_id_group)
revs_user = revs_user_join1.filter(revs_user_join1['customern']>median_user).drop('customer_id_group')

VBox()

In [53]:
median_product = 2.0
group_product = revs_user.groupBy('product_id').count().withColumnRenamed("product_id", "product_id_group").withColumnRenamed("count", "productn")
revs_product_join2 = revs_user.join(group_product, revs_user.product_id == group_product.product_id_group)
revs_product = revs_product_join2.filter(revs_product_join2['product_id']>median_product).drop('product_id_group')

VBox()

In [54]:
def find_sen_median(sen_list):
    try:
        median = np.median(sen_list) 
        return round(float(median),2)
    except Exception:
        return None

median_finder = F.udf(find_sen_median,FloatType())


VBox()

In [46]:
#find top 10 users
sentence_list = revs_product.groupBy("customer_id").agg(F.collect_list("sNum").alias("sentence_list"))
sentence_median_users = sentence_list.withColumn("median",median_finder("sentence_list"))
print(' top 10 users ranked by median number of sentences in the reviews they have published')
sentence_median_users.sort('median', ascending=False).drop('sentence_list').show(10)

VBox()

 top 10 users ranked by median number of sentences in the reviews they have published
+-----------+------+
|customer_id|median|
+-----------+------+
|   51865782| 251.0|
|   25628286| 237.5|
|   37118941| 229.0|
|   23717536| 208.0|
|   50595705| 196.0|
|   17821650| 184.0|
|   43879820| 181.0|
|   31401810| 160.0|
|   46097534| 145.0|
|   10794201| 142.0|
+-----------+------+
only showing top 10 rows

In [47]:
#find top 10 products
sentence_list1 = revs_product.groupBy("product_id").agg(F.collect_list("sNum").alias("sentence_list"))
sentence_median_products = sentence_list1.withColumn("median",median_finder("sentence_list"))
print('top 10 products ranked by median number of sentences in the reviews they have')
sentence_median_products.sort('median', ascending=False).drop('sentence_list').show(10)

VBox()

top 10 products ranked by median number of sentences in the reviews they have
+----------+------+
|product_id|median|
+----------+------+
|B00T5GY470|  89.0|
|B003Z4Y5HW|  87.0|
|B007USIWPU|  85.0|
|B0000A2ZV8|  85.0|
|B000000SI5|  78.0|
|B00061NSU2|  74.0|
|B00074XDVC|  73.0|
|B005SGZBHI|  64.0|
|B000BCE3IM|  59.0|
|B00000DS0A|  59.0|
+----------+------+
only showing top 10 rows

### Filter method 2
use the original dataframe conduct 3 filtering tasks separately. Then, find the intersection of the 3 filtered dataframe

In [ ]:
rev_sen_filter = calculate_sentence.filter("sNum > 1").cache()

In [35]:
group_user = calculate_sentence.groupBy('customer_id').count().withColumnRenamed("customer_id", "customer_id_group").withColumnRenamed("count", "customern")
revs_filter_user = calculate_sentence.join(group_user, calculate_sentence.customer_id == group_user.customer_id_group).filter('customern>1').drop('customer_id_group','customern')

VBox()

In [36]:
group_product = calculate_sentence.groupBy('product_id').count().withColumnRenamed("product_id", "product_id_group").withColumnRenamed("count", "productn")
revs_filter_product = calculate_sentence.join(group_product, calculate_sentence.product_id == group_product.product_id_group).filter('productn>2').drop('product_id_group','productn')

VBox()

In [37]:
sen_user_join = rev_sen_filter.intersect(revs_filter_user)
final_result = sen_user_join.intersect(revs_filter_product)

VBox()

In [38]:
sentence_list = final_result.groupBy("customer_id").agg(F.collect_list("sNum").alias("sentence_list"))
sentence_median_users = sentence_list.withColumn("median",median_finder("sentence_list"))
print(' top 10 users ranked by median number of sentences in the reviews they have published')
sentence_median_users.sort('median', ascending=False).drop('sentence_list').show(10)

VBox()

 top 10 users ranked by median number of sentences in the reviews they have published
+-----------+------+
|customer_id|median|
+-----------+------+
|   51865782| 251.0|
|   25628286| 237.5|
|   37118941| 229.0|
|   23717536| 208.0|
|   29580246| 201.0|
|   50595705| 196.0|
|   17821650| 184.0|
|   43879820| 181.0|
|   15585529| 180.0|
|   13255865| 163.0|
+-----------+------+
only showing top 10 rows

In [40]:
sentence_list1 = final_result.groupBy("product_id").agg(F.collect_list("sNum").alias("sentence_list"))
sentence_median_product = sentence_list1.withColumn("median",median_finder("sentence_list"))
print('top 10 products ranked by median number of sentences in the reviews they have')
sentence_median_product.sort('median', ascending=False).drop('sentence_list').show(10)

VBox()

top 10 products ranked by median number of sentences in the reviews they have
+----------+------+
|product_id|median|
+----------+------+
|B00LTQ5EVY|1784.0|
|B009SF2GZU| 327.0|
|B000003G29| 281.0|
|B00T7TYTCK| 267.5|
|B000BCH5PK| 210.0|
|B0000C0FEW| 200.0|
|B00NEX8GNI| 185.0|
|B00AP5M4WM| 164.0|
|B00004UFUK| 163.0|
|B009SF2IRG| 163.0|
+----------+------+
only showing top 10 rows